<a href="https://colab.research.google.com/github/DininduChamikara/Simple-Transformers-NER-Sinhala/blob/branch_11_04/Simple_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 93.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.2 MB/s eta

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/Research Project/ner_dataset_v3.0.csv", encoding="utf-8")
data.head()

,Sentence #,Word,Tag
0,1,ආසියානු,LOCATION
1,1,ක්‍රිකට්,OTHER
2,1,ශුර,OTHER
3,1,ශ්‍රී,LOCATION
4,1,ලංකා,LOCATION


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
data.rename(columns={"Sentence #": "sentence_id", "Word": "words", "Tag": "labels"}, inplace=True)

In [7]:
data

,sentence_id,words,labels
0,1,ආසියානු,LOCATION
1,1,ක්‍රිකට්,OTHER
2,1,ශුර,OTHER
3,1,ශ්‍රී,LOCATION
4,1,ලංකා,LOCATION
...,...,...,...
23608,713,ප්‍රවාහනය,OTHER
23609,713,කිරීමත්,OTHER
23610,713,තහනම්,OTHER
23611,713,කර,OTHER


In [8]:
X = data[["sentence_id", "words"]]
Y = data["labels"]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [10]:
# building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"], "words":x_train["words"], "labels":y_train})
test_data = pd.DataFrame({"sentence_id": x_test["sentence_id"], "words": x_test["words"], "labels": y_test})

In [11]:
train_data

,sentence_id,words,labels
7476,296,එය,OTHER
13233,463,ඒ,OTHER
10617,377,අනුව,OTHER
8371,313,පෙරහරවල්,OTHER
17899,581,සංචාරකයින්,OTHER
...,...,...,...
16835,552,නැති,OTHER
10533,374,මාර්ග,OTHER
7668,301,පවා,OTHER
22408,681,චෙක්,LOCATION


###Model Training

In [12]:
from simpletransformers.ner import NERModel, NERArgs

In [13]:
label = data['labels'].unique().tolist()
label

['LOCATION', 'OTHER', 'PERSON', 'DATE', 'ORGANIZATION', 'TIME']

In [14]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [15]:
model = NERModel('bert', 'bert-base-cased', labels=label, args=args, use_cuda= False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model.train_model(train_data, eval_data=test_data, acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/23 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/23 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/23 [00:00<?, ?it/s]

(230, 0.4787221961695215)

In [17]:
result, model_outputs, preds_list = model.eval_model(test_data)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/21 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: LOCATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PERSON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORGANIZATION seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: DATE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_

In [18]:
result

{'eval_loss': 0.5052906572818756,
 'precision': 0.6885245901639344,
 'recall': 0.10579345088161209,
 'f1_score': 0.18340611353711792}

In [19]:
prediction, model_output = model.predict(["2023-2025 ලෝක ටෙස්ට් ශූරතා තරගාවලිය එළඹෙන ජූනි මාසයේ දී ආරම්භ වීමට නියමිත යි. ඒ සඳහා සහභාගි වන එක් එක් කණ්ඩායමට හිමි වන ටෙස්ට් තරග ප්‍රමාණය මේ වන විට ප්‍රකාශයට පත් කර අවසන්. ඒ අනුව මෙවර තරගාවලිය යටතේ ද ශ්‍රී ලංකාවට ටෙස්ට් තරග 12කට ක්‍රීඩා කිරීමේ අවස්ථාව ලැබෙනවා. එයින් ටෙස්ට් තරග 6ක් ශ්‍රී ලංකාවේ ද ඉතිරි ටෙස්ට් තරග 6 විදෙස් රටවල්වල දී ද පැවැත්වීමට නියමිත යි.අවසන් වරට පැවති 2021-2023 ලෝක ටෙස්ට් ශූරතා තරගාවලියේ දී ශ්‍රී ලංකාව සැලකිය යුතු තරම් දස්කම් පෙන්වීමට සමත් වුණා. එනිසා මෙවර ලෝක ටෙස්ට් ශූරතාවේ අවසන් තරගය සඳහා පිවිසීමට ශ්‍රී ලංකාවට තිබෙන අවස්ථාව පිළිබඳව යම් විශ්ලේෂණයක් මෙලෙසින් ඔබ වෙත ගෙන එනවා."])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
prediction

[[{'2023-2025': 'DATE'},
  {'ලෝක': 'OTHER'},
  {'ටෙස්ට්': 'OTHER'},
  {'ශූරතා': 'OTHER'},
  {'තරගාවලිය': 'OTHER'},
  {'එළඹෙන': 'OTHER'},
  {'ජූනි': 'OTHER'},
  {'මාසයේ': 'OTHER'},
  {'දී': 'OTHER'},
  {'ආරම්භ': 'OTHER'},
  {'වීමට': 'OTHER'},
  {'නියමිත': 'OTHER'},
  {'යි.': 'OTHER'},
  {'ඒ': 'OTHER'},
  {'සඳහා': 'OTHER'},
  {'සහභාගි': 'OTHER'},
  {'වන': 'OTHER'},
  {'එක්': 'OTHER'},
  {'එක්': 'OTHER'},
  {'කණ්ඩායමට': 'OTHER'},
  {'හිමි': 'OTHER'},
  {'වන': 'OTHER'},
  {'ටෙස්ට්': 'OTHER'},
  {'තරග': 'OTHER'},
  {'ප්\u200dරමාණය': 'OTHER'},
  {'මේ': 'OTHER'},
  {'වන': 'OTHER'},
  {'විට': 'OTHER'},
  {'ප්\u200dරකාශයට': 'OTHER'},
  {'පත්': 'OTHER'},
  {'කර': 'OTHER'},
  {'අවසන්.': 'OTHER'},
  {'ඒ': 'OTHER'},
  {'අනුව': 'OTHER'},
  {'මෙවර': 'OTHER'},
  {'තරගාවලිය': 'OTHER'},
  {'යටතේ': 'OTHER'},
  {'ද': 'OTHER'},
  {'ශ්\u200dරී': 'OTHER'},
  {'ලංකාවට': 'OTHER'},
  {'ටෙස්ට්': 'OTHER'},
  {'තරග': 'OTHER'},
  {'12කට': 'OTHER'},
  {'ක්\u200dරීඩා': 'OTHER'},
  {'කිරීමේ': 'OTHER'},
  {'අවස්ථාව': '